<a href="https://colab.research.google.com/github/Rushabhbhagat08/Netflix-Movies-and-TV-shows-Clustering/blob/main/NETFLIX_MOVIES_AND_TV_SHOWS_CLUSTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - NETFLIX MOVIES AND TV SHOWS CLUSTERING



##### **Project Type**    - EDA/Regression/Classification/Unsupervised
##### **Contribution**    - Individual/Team
##### **Team Member**- Rushabh Anilrao Bhagat

# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


This dataset consists of tv shows and movies available on Netflix as of 2019. The dataset is collected from Flixable which is a third-party Netflix search engine.

In 2018, they released an interesting report which shows that the number of TV shows on Netflix has nearly tripled since 2010. The streaming service’s number of movies has decreased by more than 2,000 titles since 2010, while its number of TV shows has nearly tripled. It will be interesting to explore what all other insights can be obtained from the same dataset.

Integrating this dataset with other external datasets such as IMDB ratings, rotten tomatoes can also provide many interesting findings.

##In this project, you are required to do
* Exploratory Data Analysis

* Understanding what type content is available in different countries

* Is Netflix has increasingly focusing on TV rather than movies in recent years.

* Clustering similar content by matching text-based features

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
from numpy import math
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.pyplot import figure
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
from datetime import datetime

from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')

import warnings
warnings.filterwarnings('ignore')

### Dataset Loading

In [ ]:
# load Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
netflix_movies=pd.read_csv('/content/drive/MyDrive/Clustering Project/NETFLIX MOVIES AND TV SHOWS CLUSTERING (1).csv')

### Dataset First View

In [ ]:
# Dataset First Look
netflix_movies.head().transpose()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
Rows=netflix_movies.shape[0]
Columns=netflix_movies.shape[1]
print(f'The number of Rows is {Rows} and Thhe number of columns is {Columns}')

### Dataset Information

In [ ]:
# Dataset Info
netflix_movies.info()

#### Duplicate Values

In [ ]:
#Drop duplicates 
netflix_movies[netflix_movies.duplicated()]

There are no duplicated values

#### Missing Values/Null Values

In [ ]:
#Checking Null Values
netflix_movies.isnull().sum()

In [ ]:
#total null values
netflix_movies.isnull().sum().sum()

There are 3631 null values in the dataset, 2389 null values in director column, 718 null values in cast column ,507 null values in country column ,10 in date_added and 7 in rating. so we need to handle the null values

In [ ]:
#Handling Null Values
netflix_movies['cast'].fillna(value='No cast',inplace=True)
netflix_movies['country'].fillna(value=netflix_movies['country'].mode()[0],inplace=True)

In [ ]:
#'date_added' and 'rating' contains an insignificant portion of the data so we will drop them from the dataset
netflix_movies.dropna(subset=['date_added','rating'],inplace=True)

In [ ]:
#Dropping Director Column
netflix_movies.drop(['director'],axis=1,inplace=True)

In [ ]:
#again checking is there any null values are not
netflix_movies.isnull().sum()

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
netflix_movies.columns

In [ ]:
# Dataset Describe
netflix_movies.describe()

### Variables Description 

**Show_id** : Unique ID for every Movie / Tv Show

**Type** : Identifier - A Movie or TV Show

**Title** : Title of the Movie / Tv Show

**Director** : Director of the Movie

**Cast** : Actors involved in the movie / show

**Country** : Country where the movie / show was produced

**Date_added** : Date it was added on Netflix

**Release_year** : Actual Releaseyear of the movie / show

**Rating** : TV Rating of the movie / show

**Duration** : Total Duration - in minutes or number of seasons

**Listed_in** : Genere

**Description**: The Summary description

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.

### What all manipulations have you done and insights you found?

Answer Here.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
netflix_movies['type'].value_counts()

# df['type'].value_counts()
# pd.to_numeric(netflix_movies['type'], errors="ignore")
# x.astype(int)

In [ ]:
# Chart - 1 visualization code
#countplot to visualize the number of movies and tv_shows in type column
# sns.countplot(netflix_movies['type'])

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
netflix_movies['rating']

In [ ]:
#Assigning the Ratings into grouped categories
ratings = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}
netflix_movies['target_ages'] = netflix_movies['rating'].replace(ratings)

In [ ]:
# type should be a catego
netflix_movies['type'] = pd.Categorical(netflix_movies['type'])
netflix_movies['target_ages'] = pd.Categorical(netflix_movies['target_ages'], categories=['Kids', 'Older Kids', 'Teens', 'Adults'])

In [ ]:
# type should be a catego
netflix_movies['type'] = pd.Categorical(netflix_movies['type'])
netflix_movies['target_ages'] = pd.Categorical(netflix_movies['target_ages'], categories=['Kids', 'Older Kids', 'Teens', 'Adults'])

In [ ]:
netflix_movies

In [ ]:
#creating two extra columns
tv_shows=netflix_movies[netflix_movies['type']=='TV Show']
movies=netflix_movies[netflix_movies['type']=='Movie']

In [ ]:
#Rating based on rating system of all TV Shows
tv_ratings = tv_shows.groupby(['rating'])['show_id'].count().reset_index(name='count').sort_values(by='count',ascending=False)
fig_dims = (14,7)
fig, ax = plt.subplots(figsize=fig_dims)  
sns.pointplot(x='rating',y='count',data=tv_ratings)
plt.title('TV Show Ratings',size='20')
plt.show()

TV-MA has the highest number of ratings for tv shows i,e adult ratings

In [ ]:
#Movie Ratings based on Target Age Groups
plt.figure(figsize=(14,6))
plt.title('movie ratings')
sns.countplot(x=movies['rating'],hue=movies['target_ages'],data=movies,order=movies['rating'].value_counts().index)

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
movies_year =movies['release_year'].value_counts().sort_index(ascending=False)

In [ ]:
movies_year

In [ ]:
tvshows_year =tv_shows['release_year'].value_counts().sort_index(ascending=False)

In [ ]:

# visualizing the movies and tv_shows based on the release year
sns.set(font_scale=1.4)
movies_year.plot(figsize=(12, 8), linewidth=2.5, color='maroon',label="Movies / year",ms=3)
tvshows_year.plot(figsize=(12, 8), linewidth=2.5, color='blue',label="TV Shows / year")
plt.xlabel("Years", labelpad=15)
plt.ylabel("Number", labelpad=15)
plt.title("Production growth yearly", y=1.02, fontsize=22);

In [ ]:
#Analysing how many movies released per year in last 20 years
plt.figure(figsize=(15,5))
sns.countplot(y=movies['release_year'],data=netflix_movies,order=movies['release_year'].value_counts().index[0:20])

highest number of movies released in 2017 and 2018


In [ ]:
tvshows_year

In [ ]:
#Analysing how many movies released per year in last 15 years
plt.figure(figsize=(15,5))
sns.countplot(y=tv_shows['release_year'],data=netflix_movies,order=tv_shows['release_year'].value_counts().index[0:20])

In [ ]:
netflix_movies

In [ ]:
#adding columns of month and year of addition
netflix_movies['month'] = pd.DatetimeIndex(netflix_movies['date_added']).month
netflix_movies['month'].astype(int)
netflix_movies.head()    

#### Chart - 4

In [ ]:
# Chart - 5 visualization code
fig, ax = plt.subplots(figsize=(15,6))
sns.countplot(x='month', hue='type',lw=5, data=netflix_movies, ax=ax)

The above graph shows that the most content is added to Netflix from october to january

#### Chart - 5

In [ ]:

#Analysing top10 genre of the movies
plt.figure(figsize=(14,6))
plt.title('Top10 Genre of Movies',fontweight="bold")
sns.countplot(y=movies['listed_in'],data=movies,order=movies['listed_in'].value_counts().index[0:10])

#### Chart - 6

In [ ]:
# Chart - 6 visualization code
#Analysing top10 genres of TVSHOWS
plt.figure(figsize=(14,6))
plt.title('Top10 Genre of TV Shows',fontweight="bold")
sns.countplot(y=tv_shows['listed_in'],data=tv_shows,order=tv_shows['listed_in'].value_counts().index[0:10])
     

#### Chart - 7

In [ ]:
# Chart - 7 visualization code
#Checking the distribution of Movie Durations
plt.figure(figsize=(10,7))
#Regular Expression pattern \d is a regex pattern for digit + is a regex pattern for at leas
sns.distplot(movies['duration'].str.extract('(\d+)'),kde=False, color=['red'])
plt.title('Distplot with Normal distribution for Movies',fontweight="bold")
plt.show()

#### Chart - 8

In [ ]:
# Chart - 8 visualization code
#Checking the distribution of TV SHOWS
plt.figure(figsize=(30,6))
plt.title("Distribution of TV Shows duration",fontweight='bold')
sns.countplot(x=tv_shows['duration'],data=tv_shows,order = tv_shows['duration'].value_counts().index)


highest number of tv_shows consistig of single season

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

movies['minute'] = movies['duration'].str.extract('(\d+)').apply(pd.to_numeric)
duration_year = movies.groupby(['rating'])['minute'].mean()
duration_df=pd.DataFrame(duration_year).sort_values('minute')
plt.figure(figsize=(12,6))
ax=sns.barplot(x=duration_df.index, y=duration_df.minute)

Those movies that have a rating of NC-17 have the longest average duration.

When it comes to movies having a TV-Y rating, they have the shortest runtime on average

#### Chart - 10

In [ ]:
# Chart - 10 visualization code
#Analysing top15 countries with most content 
plt.figure(figsize=(18,5))
sns.countplot(x=netflix_movies['country'],order=netflix_movies['country'].value_counts().index[0:15],hue=netflix_movies['type'])
plt.xticks(rotation=50)
plt.title('Top 15 countries with most contents', fontsize=15, fontweight='bold')
plt.show()

unitated states has the highest number of content on the netflix ,followed by india

#### Chart - 11

In [ ]:
# Chart - 11 visualization code
#top_two countries where netflix is most popular
country=netflix_movies['country'].value_counts().reset_index()
country

#### Chart - 12

In [ ]:
# Chart - 12 visualization code
# Plotting the Horizontal bar plot for top 10 country contains Movie & TV Show split
country_order = netflix_movies['country'].value_counts()[:11].index
content_data = netflix_movies[['type', 'country']].groupby('country')['type'].value_counts().unstack().loc[country_order]
content_data['sum'] = content_data.sum(axis=1)
content_data_ratio = (content_data.T / content_data['sum']).T[['Movie', 'TV Show']].sort_values(by='Movie',ascending=False)[::-1]

# Plotting the barh
fig, ax = plt.subplots(1,1,figsize=(15, 8),)

ax.barh(content_data_ratio.index, content_data_ratio['Movie'], 
        color='crimson', alpha=0.8, label='Movie')
ax.barh(content_data_ratio.index, content_data_ratio['TV Show'], left=content_data_ratio['Movie'], 
        color='black', alpha=0.8, label='TV Show')

India has highest number of movies in netflix

#### Chart - 13

In [ ]:
# Chart - 13 visualization code
netflix_movies['date_added'] = pd.to_datetime(netflix_movies['date_added'])
movies['year_added'] = netflix_movies['date_added'].dt.year
netflix_movies

In [ ]:

movies['originals'] = np.where(movies['release_year'] == movies['year_added'], 'Yes', 'No')
# pie plot showing percentage of originals and others in movies
fig, ax = plt.subplots(figsize=(5,5),facecolor="#363336")
ax.patch.set_facecolor('#363336')
explode = (0, 0.1)
ax.pie(movies['originals'].value_counts(), explode=explode, autopct='%.2f%%', labels= ['Others', 'Originals'],
       shadow=True, startangle=90,textprops={'color':"black", 'fontsize': 20}, colors =['red','#F5E9F5'])

30% movies released on Netflix. 70% movies added on Netflix were released earlier by different mode.

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code
# Preparing data for heatmap
netflix_movies['count'] = 1
data = netflix_movies.groupby('country')[['country','count']].sum().sort_values(by='count',ascending=False).reset_index()[:10]
data = data['country']
df_heatmap = netflix_movies.loc[netflix_movies['country'].isin(data)]
df_heatmap = pd.crosstab(df_heatmap['country'],df_heatmap['target_ages'],normalize = "index").T
df_heatmap

In [ ]:
# Plotting the heatmap
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

country_order2 = ['United States', 'India', 'United Kingdom', 'Canada', 'Japan', 'France', 'South Korea', 'Spain',
       'Mexico']

age_order = ['Adults', 'Teens', 'Older Kids', 'Kids']

sns.heatmap(df_heatmap.loc[age_order,country_order2],cmap="YlGnBu",square=True, linewidth=2.5,cbar=False,
            annot=True,fmt='1.0%',vmax=.6,vmin=0.05,ax=ax,annot_kws={"fontsize":12})
plt.show()

the US and UK are closely aligned with their Netflix target ages, but radically different from, example, India or Japan!

Also, Mexico and Spain have similar content on Netflix for different age groups.

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

HO:movies rated for kids and older kids are at least two hours long.

H1:movies rated for kids and older kids are not at least two hours long.

In [ ]:
movies

In [ ]:
#making copy of df_clean_frame
df_hypothesis=netflix_movies.copy()
#head of df_hypothesis
df_hypothesis.head()
     

In [ ]:
#filtering movie from Type_of_show column
df_hypothesis = df_hypothesis[df_hypothesis["type"] == "Movie"]

In [ ]:

#with respect to each ratings assigning it into group of categories                 
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}

df_hypothesis['target_ages'] = df_hypothesis['rating'].replace(ratings_ages)
#let's see unique target ages 
df_hypothesis['target_ages'].unique()

In [ ]:
#Another category is target_ages (4 classes).
df_hypothesis['target_ages'] = pd.Categorical(df_hypothesis['target_ages'], categories=['Kids', 'Older Kids', 'Teens', 'Adults'])
#from duration feature extractin string part and after extracting Changing the object type to numeric
df_hypothesis['duration']= df_hypothesis['duration'].str.extract('(\d+)')
df_hypothesis['duration'] = pd.to_numeric(df_hypothesis['duration'])
#head of df_
df_hypothesis.head(3)


In [ ]:

#group_by duration and target_ages                 
group_by_= df_hypothesis[['duration','target_ages']].groupby(by='target_ages')
#mean of group_by variable
group=group_by_.mean().reset_index()
group
     

In [ ]:

#In A and B variable grouping values 
A= group_by_.get_group('Kids')
B= group_by_.get_group('Older Kids')
#mean and std. calutation for kids and older kids variables
M1 = A.mean()
S1 = A.std()

M2= B.mean()
S2 = B.std()

print('Mean for movies rated for Kids {} \n Mean for  movies rated for older kids {}'.format(M1,M2))
print('Std for  movies rated for Older Kids {} \n Std for  movies rated for kids {}'.format(S2,S1))

In [ ]:
#import stats 
from scipy import stats
#length of groups and DOF
n1 = len(A)
n2= len(B)
print(n1,n2)

dof = n1+n2-2
print('dof',dof)

sp_2 = ((n2-1)*S1**2  + (n1-1)*S2**2) / dof
print('SP_2 =',sp_2)

sp = np.sqrt(sp_2)
print('SP',sp)

#tvalue
t_val = (M1-M2)/(sp * np.sqrt(1/n1 + 1/n2))
print('tvalue',t_val[0])

In [ ]:
#t-distribution
stats.t.ppf(0.025,dof)

In [ ]:
#t-distribution
stats.t.ppf(0.975,dof)

Because the t-value is not in the range, the null hypothesis is rejected.

As a result, movies rated for kids and older kids are not at least two hours long.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

H1:The duration which is more than 90 mins are movies

HO:The duration which is more than 90 mins are NOT movies

#### 2. Perform an appropriate statistical test.

In [ ]:

#making copy of df_clean_frame
df_hypothesis=netflix_movies.copy()
#head of df_hypothesis
df_hypothesis.head()

In [ ]:

df_hypothesis['duration']= df_hypothesis['duration'].str.extract('(\d+)')
df_hypothesis['duration'] = pd.to_numeric(df_hypothesis['duration'])

In [ ]:

df_hypothesis['type'] = pd.Categorical(df_hypothesis['type'], categories=['Movie','TV Show'])
df_hypothesis.head(3)

In [ ]:
#group_by duration and TYPE                 
group_by_= df_hypothesis[['duration','type']].groupby(by='type')
#mean of group_by variable
group=group_by_.mean().reset_index()
group

In [ ]:
#In A and B variable grouping values 
A= group_by_.get_group('Movie')
B= group_by_.get_group('TV Show')
#mean and std
M1 = A.mean()
S1 = A.std()

M2= B.mean()
S2 = B.std()

print('Mean  {}'.format(M1,M2))
print('Std  {}'.format(S2,S1))

In [ ]:
#import stats 
from scipy import stats
#length of groups and DOF
n1 = len(A)
n2= len(B)
print(n1,n2)

dof = n1+n2-2
print('dof',dof)

sp_2 = ((n2-1)*S1**2  + (n1-1)*S2**2) / dof
print('SP_2 =',sp_2)

sp = np.sqrt(sp_2)
print('SP',sp)

#tvalue
t_val = (M1-M2)/(sp * np.sqrt(1/n1 + 1/n2))
print('tvalue',t_val[0])

In [ ]:
#t-distribution
stats.t.ppf(0.025,dof)


In [ ]:
#t-distribution
stats.t.ppf(0.975,dof) 

Because the t-value is not in the range, the null hypothesis is rejected.

As a result, The duration which is more than 90 mins are movies

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# show the datatpe of dataaset
netflix_movies.dtypes

In [ ]:
netflix_movies['description'].astype(str)

### 2. Feature Manipulation & Selection

In [ ]:
# after above all the changes, those features are in list format, so making list of description feature
netflix_movies['description'] = netflix_movies['description'].apply(lambda x: x.split(' '))

In [ ]:
# converting text feature to string from list
netflix_movies['description']= netflix_movies['description'].apply(lambda x: " ".join(x))
# making all the words in text feature to lowercase
netflix_movies['description']= netflix_movies['description'].apply(lambda x: x.lower())

#### 3. Removing Punctuations

In [ ]:
# removing punctution 
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)
# applying above function on text feature
netflix_movies['description']= netflix_movies['description'].apply(remove_punctuation)

In [ ]:
# descreption of 10 rows
netflix_movies['description'][0:10]

#### 4. Removing Stopwords & Removing White spaces

In [ ]:
# using nltk library to download stopwords
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
sw=stopwords.words('english')
#Defining stopwords 
def stopwords(text):
    '''a function for removing the stopword'''
    text = [word for word in text.split() if word not in sw]
    # joining the list of words with space separator
    return " ".join(text)
# applying above function on text feature
netflix_movies['description']=netflix_movies['description'].apply(stopwords)
# this is how value in text looks like after removing stopwords
netflix_movies['description'][0]

In [ ]:
# importing TfidVectorizer from sklearn library
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Applying Tfidf Vectorizer
tfidfmodel = TfidfVectorizer(max_features=5000)
X_tfidf = tfidfmodel.fit_transform(netflix_movies['description'])
X_tfidf.shape

In [ ]:
# convert X into array form for clustering
X = X_tfidf.toarray() 

## ***7. ML Model Implementation***

### ML Model - 1

Finding the optimal number of clusters using the elbow method

In [ ]:
# ML Model - 1 Implementation
#finding optimal number of clusters using the elbow method  
from sklearn.cluster import KMeans  
wcss_list= []  #Initializing the list for the values of WCSS  
  
#Using for loop for iterations from 1 to 30.  
for i in range(1, 30):  
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state= 42)  
    kmeans.fit(X)  
    wcss_list.append(kmeans.inertia_)  
plt.plot(range(1, 30), wcss_list)  
plt.title('The Elobw Method Graph')  
plt.xlabel('Number of clusters(k)')  
plt.ylabel('wcss_list')  
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
#sillhoute score of clusters 
sill = [] 
for i in range(2,30):
    model = KMeans(n_clusters=i,init ='k-means++',random_state=51)
    model.fit(X)
    y1 = model.predict(X)
    score = silhouette_score(X,y1)
    sill.append(score)
    print('cluster: %d \t Sillhoute: %0.4f'%(i,score))

In [ ]:
#Plotting Sillhoute's score
plt.plot(sill,'bs--')
plt.xticks(list(range(0,30)),list(range(2,32)))
plt.grid(),plt.xlabel('Number of cluster')
plt.show()

In [ ]:
#training the K-means model on a dataset  
kmeans = KMeans(n_clusters= 26, init='k-means++', random_state= 42)  
y_predict= kmeans.fit_predict(X) 

#evalution

In [ ]:
#Predict the clusters and evaluate the silhouette score
score = silhouette_score(X, y_predict)
print("Silhouette score is {}".format(score))

In [ ]:
#davies_bouldin_score of our clusters 
davies_bouldin_score(X, y_predict)

In [ ]:
#Adding a seperate column for the cluster
netflix_movies["cluster"] = y_predict

In [ ]:
netflix_movies['cluster'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.countplot(x='cluster', hue='type',lw=5, data=netflix_movies, ax=ax)

In [ ]:
#SCATTER PLOT FOR CLUSTERS
fig = px.scatter(df, y="description", x="cluster",color="cluster")
fig.update_traces(marker_size=100)
fig.show()

### ML Model - 2

In [ ]:
plt.figure(figsize =(8, 8))
plt.title('Visualising the data')
Dendrogram = shc.dendrogram((shc.linkage(X, method ='ward')))

### ML Model - 3

In [ ]:
#Fitting our variable in Agglomerative Clusters
aggh = AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='ward')  
aggh.fit(X)
#Predicting using our model
y_hc=aggh.fit_predict(X)

In [ ]:
df_hierarchical =netflix_movies.copy()
#creating a column where each row is assigned to their separate cluster
df_hierarchical['cluster'] = aggh.labels_
df_hierarchical.head()

##Evalution

In [ ]:
#Silhouette Coefficient
print("Silhouette Coefficient: %0.3f"%silhouette_score(X,y_hc, metric='euclidean'))

In [ ]:
#davies_bouldin_score of our clusters 
davies_bouldin_score(X, y_hc)     

# **Conclusion**

* from elbow and sillhoute score ,optimal of 26 clusters formed , K Means is best for identification than Hierarchical as the evaluation metrics also indicates the same.in kmean cluster 0 has the highest number of datapoints
and evnly distributed for other cluster.

* Netflix has 5372 movies and 2398 TV shows,
there are more number movies on Netflix than TV shows.

* TV-MA has the highest number of ratings for tv shows i,e adult ratings

* highest number of movies released in 2017 and 2018.

* highest number of movies released in 2020 The number of movies on Netflix is growing significantly faster than the number of TV shows. We saw a huge increase in the number of movies and television episodes after 2015. there is a significant drop in the number of movies and television episodes produced after 2020. It appears that Netflix has focused more attention on increasing Movie content than TV Shows. Movies have increased much more dramatically than TV shows.

* the most content is added to Netflix from october to january

* Documentaries are the top most genre in netflix which is fllowed by standup comedy and Drams and international movies

* kids tv is the top most TV show genre in netflix

* most of the movies have duration of between 50 to 150

* highest number of tv_shows consistig of single season

* Those movies that have a rating of NC-17 have the longest average duration.

* When it comes to movies having a TV-Y rating, they have the shortest runtime on average

* unitated states has the highest number of content on the netflix ,followed by india

* india has highest number of movies in netflix

* 30% movies released on Netflix.

* 70% movies added on Netflix were released earlier by different mode.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***